In [1]:
import datasets
from datasets import load_dataset, load_from_disk, DatasetDict, concatenate_datasets
import numpy as np
from scipy.sparse.csgraph import floyd_warshall
from transformers import DataCollatorWithPadding
from torch.utils.data import DataLoader

In [17]:
config_list = ['en_gum', 'en_lines', 'en_partut', 'en_pronouns', 'en_pud']
dataset = DatasetDict()

dataset['train'] = load_dataset('universal_dependencies', 'en_ewt', split='train')
dataset['validation'] = load_dataset('universal_dependencies', 'en_ewt', split='validation')
dataset['test'] = load_dataset('universal_dependencies', 'en_ewt', split='test')

for config in config_list:
    data = load_dataset('universal_dependencies', config)
    keys = data.keys()
    for key in keys:
        dataset[key] = concatenate_datasets([dataset[key], data[key]])

Found cached dataset universal_dependencies (/users/ujan/.cache/huggingface/datasets/universal_dependencies/en_ewt/2.7.0/1ac001f0e8a0021f19388e810c94599f3ac13cc45d6b5b8c69f7847b2188bdf7)
Found cached dataset universal_dependencies (/users/ujan/.cache/huggingface/datasets/universal_dependencies/en_ewt/2.7.0/1ac001f0e8a0021f19388e810c94599f3ac13cc45d6b5b8c69f7847b2188bdf7)
Found cached dataset universal_dependencies (/users/ujan/.cache/huggingface/datasets/universal_dependencies/en_ewt/2.7.0/1ac001f0e8a0021f19388e810c94599f3ac13cc45d6b5b8c69f7847b2188bdf7)
Found cached dataset universal_dependencies (/users/ujan/.cache/huggingface/datasets/universal_dependencies/en_gum/2.7.0/1ac001f0e8a0021f19388e810c94599f3ac13cc45d6b5b8c69f7847b2188bdf7)


  0%|          | 0/3 [00:00<?, ?it/s]

Found cached dataset universal_dependencies (/users/ujan/.cache/huggingface/datasets/universal_dependencies/en_lines/2.7.0/1ac001f0e8a0021f19388e810c94599f3ac13cc45d6b5b8c69f7847b2188bdf7)


  0%|          | 0/3 [00:00<?, ?it/s]

Found cached dataset universal_dependencies (/users/ujan/.cache/huggingface/datasets/universal_dependencies/en_partut/2.7.0/1ac001f0e8a0021f19388e810c94599f3ac13cc45d6b5b8c69f7847b2188bdf7)


  0%|          | 0/3 [00:00<?, ?it/s]

Found cached dataset universal_dependencies (/users/ujan/.cache/huggingface/datasets/universal_dependencies/en_pronouns/2.7.0/1ac001f0e8a0021f19388e810c94599f3ac13cc45d6b5b8c69f7847b2188bdf7)


  0%|          | 0/1 [00:00<?, ?it/s]

Found cached dataset universal_dependencies (/users/ujan/.cache/huggingface/datasets/universal_dependencies/en_pud/2.7.0/1ac001f0e8a0021f19388e810c94599f3ac13cc45d6b5b8c69f7847b2188bdf7)


  0%|          | 0/1 [00:00<?, ?it/s]

In [18]:
dataset

DatasetDict({
    train: Dataset({
        features: ['idx', 'text', 'tokens', 'lemmas', 'upos', 'xpos', 'feats', 'head', 'deprel', 'deps', 'misc'],
        num_rows: 21787
    })
    validation: Dataset({
        features: ['idx', 'text', 'tokens', 'lemmas', 'upos', 'xpos', 'feats', 'head', 'deprel', 'deps', 'misc'],
        num_rows: 3974
    })
    test: Dataset({
        features: ['idx', 'text', 'tokens', 'lemmas', 'upos', 'xpos', 'feats', 'head', 'deprel', 'deps', 'misc'],
        num_rows: 5440
    })
})

In [21]:
dataset['test'][56]

{'idx': 'weblog-blogspot.com_aggressivevoicedaily_20060629164800_ENG_20060629_164800-0008',
 'text': 'ALITO filed a dissenting opinion, in which SCALIA and THOMAS joined as to Parts I through III.',
 'tokens': ['ALITO',
  'filed',
  'a',
  'dissenting',
  'opinion',
  ',',
  'in',
  'which',
  'SCALIA',
  'and',
  'THOMAS',
  'joined',
  'as',
  'to',
  'Parts',
  'I',
  'through',
  'III',
  '.'],
 'lemmas': ['ALITO',
  'file',
  'a',
  'dissent',
  'opinion',
  ',',
  'in',
  'which',
  'SCALIA',
  'and',
  'THOMAS',
  'join',
  'as',
  'to',
  'part',
  'i',
  'through',
  'iii',
  '.'],
 'upos': [10, 16, 8, 16, 0, 1, 2, 11, 10, 9, 10, 16, 2, 2, 0, 3, 2, 3, 1],
 'xpos': ['NNP',
  'VBD',
  'DT',
  'VBG',
  'NN',
  ',',
  'IN',
  'WDT',
  'NNP',
  'CC',
  'NNP',
  'VBD',
  'IN',
  'IN',
  'NNS',
  'CD',
  'IN',
  'CD',
  '.'],
 'feats': ["{'Number': 'Sing'}",
  "{'Mood': 'Ind', 'Tense': 'Past', 'VerbForm': 'Fin'}",
  "{'Definite': 'Ind', 'PronType': 'Art'}",
  "{'VerbForm': 'Ger'}",
 

In [16]:
concatenate_datasets([dataset['train'], data['train']])

Dataset({
    features: ['idx', 'text', 'tokens', 'lemmas', 'upos', 'xpos', 'feats', 'head', 'deprel', 'deps', 'misc'],
    num_rows: 16830
})

In [35]:
'train' in dataset.keys()

False

In [14]:
from transformers import AutoTokenizer, XLMRobertaModel

tokenizer = AutoTokenizer.from_pretrained('xlm-roberta-base')
#model = XLMRobertaModel.from_pretrained('xlm-roberta-base')

In [10]:
dataset

DatasetDict({
    test: Dataset({
        features: ['idx', 'text', 'tokens', 'lemmas', 'upos', 'xpos', 'feats', 'head', 'deprel', 'deps', 'misc'],
        num_rows: 1000
    })
})

In [17]:
tokens = dataset['test'][10]['tokens']

In [20]:
model_inputs = tokenizer(
    tokens, truncation=True,
    max_length=256,
    is_split_into_words=True)
len(model_inputs['input_ids'])

29

In [26]:
l = [id for id in model_inputs.word_ids(0) if id is not None]
len(set(l))

22

In [12]:
len(dataset['test'][10]['head'])

22

In [13]:
len(dataset['test'][10]['tokens'])

22

In [95]:
dataset = load_from_disk('/users/ujan/linguistic-structures/data/processed/UD/en_pud_node_distance')
dataset

DatasetDict({
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'true_dist'],
        num_rows: 993
    })
})

In [105]:
type(dataset['test'][0]['input_ids'][-1])

int

In [112]:
import numpy as np
a = [1,2,3]
np.array(a)

array([1, 2, 3])

In [80]:
type(dataset['test'][0]['true_dist'])

list

In [81]:
len(dataset['test'][1]['true_dist'])

324

In [82]:
len(dataset['test'][0]['input_ids'])

47

In [83]:
type(dataset['test'][0]['input_ids'])

list

In [84]:
len(dataset['test'][1]['input_ids'])

23

In [90]:
type(dataset['test'][1]['input_ids'])

list

In [91]:
type(dataset['test'][1]['true_dist'])

list

In [67]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, padding=True, return_tensors='pt')

# data loader
dataloader = DataLoader(
    dataset["test"],
    collate_fn=data_collator,
    batch_size=16,
)

#batch = next(iter(dataloader))

In [68]:
batch = next(iter(dataloader))

You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


ValueError: Unable to create tensor, you should probably activate truncation and/or padding with 'padding=True' 'truncation=True' to have batched tensors with the same length. Perhaps your features (`true_dist` in this case) have excessive nesting (inputs type `list` where type `int` is expected).

In [77]:
dataset['test'][690]['true_dist']

[0.0,
 1.0,
 2.0,
 3.0,
 4.0,
 5.0,
 5.0,
 4.0,
 5.0,
 5.0,
 4.0,
 5.0,
 1.0,
 0.0,
 1.0,
 2.0,
 3.0,
 4.0,
 4.0,
 3.0,
 4.0,
 4.0,
 3.0,
 4.0,
 2.0,
 1.0,
 0.0,
 1.0,
 2.0,
 3.0,
 3.0,
 2.0,
 3.0,
 3.0,
 2.0,
 3.0,
 3.0,
 2.0,
 1.0,
 0.0,
 1.0,
 2.0,
 2.0,
 1.0,
 2.0,
 2.0,
 1.0,
 2.0,
 4.0,
 3.0,
 2.0,
 1.0,
 0.0,
 3.0,
 3.0,
 2.0,
 3.0,
 3.0,
 2.0,
 3.0,
 5.0,
 4.0,
 3.0,
 2.0,
 3.0,
 0.0,
 2.0,
 1.0,
 4.0,
 4.0,
 3.0,
 4.0,
 5.0,
 4.0,
 3.0,
 2.0,
 3.0,
 2.0,
 0.0,
 1.0,
 4.0,
 4.0,
 3.0,
 4.0,
 4.0,
 3.0,
 2.0,
 1.0,
 2.0,
 1.0,
 1.0,
 0.0,
 3.0,
 3.0,
 2.0,
 3.0,
 5.0,
 4.0,
 3.0,
 2.0,
 3.0,
 4.0,
 4.0,
 3.0,
 0.0,
 2.0,
 1.0,
 2.0,
 5.0,
 4.0,
 3.0,
 2.0,
 3.0,
 4.0,
 4.0,
 3.0,
 2.0,
 0.0,
 1.0,
 2.0,
 4.0,
 3.0,
 2.0,
 1.0,
 2.0,
 3.0,
 3.0,
 2.0,
 1.0,
 1.0,
 0.0,
 1.0,
 5.0,
 4.0,
 3.0,
 2.0,
 3.0,
 4.0,
 4.0,
 3.0,
 2.0,
 2.0,
 1.0,
 0.0]

In [35]:
for x in dataset1['test']:
    head = x['head']
    for h in head:
        if h.isdigit():
            if len(head) == int(h):
                print(head)
                

['13', '13', '4', '13', '6', '13', '9', '9', '13', '12', '12', '9', '0']
['13', '13', '4', '13', '6', '13', '9', '9', '13', '12', '12', '9', '0']
['13', '13', '4', '13', '6', '13', '9', '9', '13', '12', '12', '9', '0']
['13', '13', '4', '13', '6', '13', '9', '9', '13', '12', '12', '9', '0']
['13', '13', '4', '13', '6', '13', '9', '9', '13', '12', '12', '9', '0']
['2', '0', '4', '2', '6', '4']


In [40]:
dataset = dataset.filter(lambda example: all(h.isdigit() for h in example['head']))

  0%|          | 0/1 [00:00<?, ?ba/s]

In [41]:
for x in dataset['test']:
    head = x['head']
    for h in head:
        if not h.isdigit():
            #print('not digit')
            #print(h)
            print(head)
            print(len(head))
            break

In [49]:
dataset = dataset.shuffle(seed=42)

In [50]:
def distance_map(batch):
    ids = batch['idx']
    tokens = batch['tokens']
    text = batch['text']
    heads = batch['head']
    
    
    batch_size = len(heads)
    for b in range(batch_size):
        head_list = heads[b]
        print(head_list)
        seq_len = len(head_list)
        print(seq_len)
        graph = np.zeros((seq_len, seq_len))
        # populate adjacency matrix
        for s in range(seq_len):
            if head_list[s] != 0:
                graph[s, int(head_list[s])-1] = 1
        print('graph')
        print(graph)
        dist_matrix = floyd_warshall(csgraph=graph, directed=False, unweighted=True)
        print('dist matrix')
        print(dist_matrix)
        raise

    # compute adj matrix
    
dataset.map(distance_map, batched=True, batch_size=4)

  0%|          | 0/249 [00:00<?, ?ba/s]

['2', '10', '5', '5', '2', '9', '9', '9', '2', '0', '13', '13', '10', '17', '17', '17', '13', '23', '21', '21', '23', '23', '10', '10']
24
graph
[[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0

RuntimeError: No active exception to reraise

In [48]:
graph

array([[0., 0., 0.],
       [0., 3., 0.],
       [0., 0., 0.]])

In [24]:
a = np.array([
    [0, 1, 0, 0, 0],
    [1, 0, 1, 1, 0],
    [0, 1, 0, 0, 0],
    [0, 1, 0, 0, 1],
    [0, 0, 0, 1, 0]
])

a

array([[0, 1, 0, 0, 0],
       [1, 0, 1, 1, 0],
       [0, 1, 0, 0, 0],
       [0, 1, 0, 0, 1],
       [0, 0, 0, 1, 0]])

In [25]:
dist_matrix = floyd_warshall(csgraph=a, directed=False, unweighted=True)
dist_matrix

array([[0., 1., 2., 2., 3.],
       [1., 0., 1., 1., 2.],
       [2., 1., 0., 2., 3.],
       [2., 1., 2., 0., 1.],
       [3., 2., 3., 1., 0.]])

In [21]:
dist_matrix = floyd_warshall(csgraph=graph, directed=False)
dist_matrix

array([[0., 1., 2., 2., 3.],
       [1., 0., 1., 1., 2.],
       [2., 1., 0., 2., 3.],
       [2., 1., 2., 0., 1.],
       [3., 2., 3., 1., 0.]])

In [14]:
graph = np.zeros((3, 3))
graph[1,1] = 3
graph

array([[0., 0., 0.],
       [0., 3., 0.],
       [0., 0., 0.]])

In [57]:
a = np.ones(3)
b = np.zeros(3)
l = []
l.append(a)
l.append(b)
l

[array([1., 1., 1.]), array([0., 0., 0.])]

In [58]:
np.array(l)

array([[1., 1., 1.],
       [0., 0., 0.]])

In [59]:
np.array(l).shape

(2, 3)

In [2]:
import numpy as np
from scipy import stats

In [3]:
res = stats.spearmanr([1, 2, 3, 4, 5], [5, 6, 7, 8, 7])
res.statistic

0.8207826816681233

In [10]:
b = 4
s = 30

In [27]:
preds = np.random.rand(b,s,s)
labels = np.random.rand(b,s,s)

for s_i in range(s):
    res = stats.spearmanr(preds[0][s_i], labels[0][s_i], axis=1)
    print(res)

SignificanceResult(statistic=0.07630700778642936, pvalue=0.6885846533599633)
SignificanceResult(statistic=0.11546162402669634, pvalue=0.5434694510154613)
SignificanceResult(statistic=0.14616240266963293, pvalue=0.44088472389295175)
SignificanceResult(statistic=0.012235817575083427, pvalue=0.9488325740980201)
SignificanceResult(statistic=-0.1474972191323693, pvalue=0.43667732375833057)
SignificanceResult(statistic=0.06251390433815351, pvalue=0.7427795641677637)
SignificanceResult(statistic=-0.22358175750834258, pvalue=0.23495949078364692)
SignificanceResult(statistic=0.10478309232480533, pvalue=0.5815962537668229)
SignificanceResult(statistic=-0.3446051167964405, pvalue=0.06220090340541729)
SignificanceResult(statistic=-0.20978865406006672, pvalue=0.2658489717443996)
SignificanceResult(statistic=0.217797552836485, pvalue=0.24760667595245606)
SignificanceResult(statistic=0.3041156840934371, pvalue=0.10228067785201178)
SignificanceResult(statistic=0.07897664071190211, pvalue=0.67825814598

In [26]:
res.statistic

0.2703003337041157

In [23]:
res.statistic[1]

array([-0.38064516,  1.        ,  0.30055617, -0.08787542,  0.17775306,
       -0.12614016,  0.04382647,  0.13770857, -0.03804227, -0.36462736,
        0.23604004, -0.41935484,  0.00333704, -0.243604  , -0.11813126,
        0.3054505 , -0.04338154,  0.08209121, -0.04382647,  0.13414905,
        0.0028921 ,  0.14660734,  0.08164627,  0.04204672, -0.29477197,
       -0.22669633,  0.08520578, -0.08921023, -0.16974416, -0.33793103,
       -0.17686318,  0.1621802 ,  0.11101224, -0.36462736, -0.06028921,
       -0.04605117, -0.00111235, -0.10077864,  0.10211346, -0.1243604 ,
        0.11590656,  0.13192436, -0.02157953,  0.22358176, -0.14883204,
        0.34638487, -0.17730812,  0.06918799,  0.01802002, -0.05094549,
        0.1461624 ,  0.06206897, -0.05183537,  0.00912125, -0.2080089 ,
        0.02335929, -0.08342603,  0.18086763, -0.0625139 , -0.07007786])